In [91]:
import pandas as pd
import numpy as np
import os, gc
os.chdir("/home/fvalle/phd/datasets/gtex/log/3000_random-5000hvg/")

In [98]:
df=pd.read_csv("mainTable.csv", index_col=0)

In [99]:
with open("corpus.txt",'w') as file:
    for sample in df.columns:
        for g in np.array(df[sample].sort_values(ascending=False).index[:1000],dtype=str):
            file.write(g)
            file.write(" ")
        file.write("\n")

In [100]:
gc.collect()

8

In [101]:
os.system("topicmap -f corpus.txt -r 10 -t 500 -o tm")

65280

In [102]:
df_words = pd.read_csv("tm/word_wn_count.txt", sep=' ', header=None)
df_words.columns=['word', 'word-id', 'occurrence']
df_words.sort_values('word-id')

,word,word-id,occurrence
1331,ENSG00000131095,0,633
2923,ENSG00000087086,1,3000
422,ENSG00000101405,2,101
2574,ENSG00000154277,3,2314
2762,ENSG00000244734,4,2683
...,...,...,...
22,ENSG00000253333,2933,1
23,ENSG00000247473,2934,1
24,ENSG00000232347,2935,1
25,ENSG00000232270,2936,1


In [103]:
df_topic_distr = pd.read_csv("tm/lda_gammas_final.txt", sep=' ', header=None)
df_topic_distr.columns=['Topic %d'%(t+1) for t in df_topic_distr.columns]
df_topic_distr.index.name='i_doc'
df_topic_distr.insert(0,'doc',df.columns)
df_topic_distr=df_topic_distr.dropna(how='all',axis=1)
df_topic_distr

,doc,Topic 1,Topic 2
i_doc,,,
0,GTEX-16NPV-0011-R8a-SM-7EPH8,1000.070000,0.143525
1,GTEX-U8T8-0126-SM-3DB94,0.065732,1000.140000
2,GTEX-12ZZW-0011-R5a-SM-5DUVN,1000.070000,0.143525
3,GTEX-11TT1-1926-SM-5PNYN,0.065732,1000.140000
4,GTEX-T6MN-1226-SM-3NMA5,264.569000,735.640000
...,...,...,...
2995,GTEX-13FTZ-2226-SM-5LZXI,244.618000,755.591000
2996,GTEX-1GL5R-2026-SM-7MKHP,231.219000,768.991000
2997,GTEX-ZTPG-0426-SM-5O98K,48.168700,952.041000


In [104]:
clusters = df_topic_distr.drop('doc',1).values.argmax(1)
df_clusters=pd.DataFrame()
for cluster in range(np.max(clusters)+1):
    elems=df.columns[clusters==cluster].values
    df_clusters.insert(0,'Cluster %d'%(cluster+1),np.concatenate([elems, ['' for _ in range(len(df.columns)- len(elems))]]))

In [105]:
df_clusters.sort_index(axis=1).to_csv("tm/tm_level_0_clusters.csv", index=False, header=True)

In [106]:
df_topic_distr.to_csv("tm/tm_level_0_topic-dist.csv", index=True, header=True)

In [107]:
df_word_distr = pd.DataFrame().fillna(0)
with open("tm/lda_betas_sparse_final.txt","r") as f:
    for line in f.read().split("\n"):
        row = line.split(" ")
        if len(row) < 2:
            continue
        topic = int(row[0])+1
        line=np.array(row[1:-1], dtype=float).reshape(int((len(row)-1)/2),2)
        for el in line:
            df_word_distr.at[df_words[df_words['word-id']==int(el[0])].word.values[0], f"Topic {topic}"] = el[1]
#df_word_distr.index=df_words['word']
df_word_distr.fillna(0)
df_word_distr.to_csv("tm/tm_level_0_word-dist.csv", index=True, header=True)